In [ ]:
from helper_functions import *

### Figure 1 ###

frozen_lake_colors = ['#043765', "#edf8fe"]
meat_colors = ['#d22c2c', '#f4d5ce']
rainbow_colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'pink', 'cyan', 'magenta', 'navy']

ModuleNotFoundError: No module named 'helper_functions'